<a href="https://colab.research.google.com/github/spaziochirale/EsperimentiVari/blob/main/rag_chatbot_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prototipo di ChatBot RAG con LangChain e PDF

Questo notebook mostra come realizzare un *chatBot* di tipo **RAG** utilizzando il framework **LangChain** e un file PDF come base di conoscenza.

Per prima cosa installiamo sul server alcuni package Python della piattaforma LangChain e altre librerie necessarie che utilizzeremo nel prototipo.

In [ ]:
!pip install -qU langchain langchain_community langchain_chroma pypdf

Installiamo il modulo specifico di LangChain che interfaccia le API di OpenAI.

In [ ]:
!pip install -qU langchain-openai

Impostiamo la chiave API di OpenAI e creiamo l'oggetto `llm`.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Importiamo le librerie necessarie per il nostro progetto.

In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

Carichiamo il file PDF. Nota: devi caricare un file PDF su Colab prima di eseguire questa cella.

In [ ]:
from google.colab import files
uploaded = files.upload()

pdf_file = list(uploaded.keys())[0]  # Prende il nome del primo file caricato
loader = PyPDFLoader(pdf_file)
docs = loader.load()

Dividiamo il contenuto del PDF in chunks e creiamo il database vettoriale.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

Creiamo il retriever e carichiamo il prompt template.

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

Definiamo una funzione di utilità per formattare i documenti.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Definiamo la chain RAG.

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Ora possiamo utilizzare la chain RAG per porre domande sul contenuto del PDF.

In [ ]:
question = "Cosa fa il timer di stand-by?"
rag_chain.invoke(question)

In [ ]:
retriever.invoke("Cosa fa il timer di stand-by?")